In [1]:
import pandas as pd
import numpy as np
import researchpy # pip install researchpy
import statistics

import plotly as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)
import plotly.io as pio
%config InlineBackend.figure_format = 'svg'

carrierDF=pd.read_csv('../data/allCarrierData_final.csv')
carrierDF.Manufacture=carrierDF.Manufacture.str.replace("Samgung","Samsung")
carrierDF.Manufacture=carrierDF.Manufacture.str.replace("KYOCERA","Kyocera")
carrierDF.Carrier=carrierDF.Carrier.str.replace("TMobile","T-Mobile")
print(carrierDF.head())

  Carrier Manufacture               Model         Release_Date  \
0    AT&T     Alcatel  Alcatel IdealXCITE  2018-04-03 00:00:00   
1    AT&T     Alcatel  Alcatel IdealXCITE  2018-04-03 00:00:00   
2    AT&T     Alcatel  Alcatel IdealXCITE  2018-10-24 00:00:00   
3    AT&T     Alcatel  Alcatel IdealXCITE  2018-12-18 00:00:00   
4    AT&T  BlackBerry   BlackBerry KEYone  2017-09-26 00:00:00   

        Bulletin_Level  difference Android_Level_Guessed  \
0  2018-02-01 00:00:00          61                 False   
1  2018-04-01 00:00:00           2                  True   
2  2018-08-01 00:00:00          84                 False   
3  2018-10-01 00:00:00          78                 False   
4  2017-09-01 00:00:00          25                 False   

                                               build  Year_Phone_Release  \
0        JO.2.0.c1-00150-8909_GEN_PACK-1 nan YAMUAM0                2017   
1                                nan YB4UA40File nan                2017   
2        JO.2.

In [2]:
# Variables for looping
uniqueCarriers=carrierDF.Carrier.unique()
uniqueManufactures=carrierDF.Manufacture.unique()
uniqueModels=carrierDF.Model.unique()
uniqueBulletins=carrierDF.Bulletin_Level.unique()



lvlColor={}
carrierColors={'AT&T':[64, 190, 245],'Sprint':[255,162,0],'T-Mobile':[255,0,238],'Verizon':[255,21,0]}
print(uniqueCarriers)
for index,carrier in enumerate(uniqueCarriers):
    if not carrier in lvlColor:
        #randx=random.randint(0,255)
        #randy=random.randint(0,255)
        #randz=random.randint(0,255)
        #lvlColor[carrier]=(randx, randy, randz)
        lvlColor[carrier]=(carrierColors[carrier][0], carrierColors[carrier][1], carrierColors[carrier][2])

manufactureColors={'BlackBerry':[38, 38, 38],'HTC':[189, 15, 108],'LG':[84, 201, 240],'Motorola':[1, 11, 120],
                   'Samsung':[95, 201, 125],'Google':[137, 214, 60],'Huawei':[191, 11, 11],
                   'Kyocera':[245, 118, 118],'ZTE':[102, 255, 247],'Essential':[20, 10, 46],
                   'ASUS':[12, 55, 59],'OnePlus':[227, 34, 73],'Alcatel':[0, 188, 217],
                   'RED':[219, 0, 0],'Razer':[219, 0, 0],'CAT':[255, 179, 0],'Coolpad':[237, 150, 0],
                   'Slate':[50, 0, 61],"Sonim":[219, 0, 22],"Sony":[23, 23, 23],'Revvl(T-Mobile)':[255,0,238],
                   "Nokia":[14, 0, 171],'Orbic':[139, 214, 0],"Palm":[255,21,0]}
print(uniqueManufactures)
# colors to work with: https://www.google.com/search?q=rgb+color+picker&oq=rgb+colo&aqs=chrome.0.69i59j0j69i57j0l3.1905j1j7&sourceid=chrome&ie=UTF-8
for index,manufacture in enumerate(uniqueManufactures):
    if not manufacture in lvlColor:
        #randx=random.randint(0,255)
        #randy=random.randint(0,255)
        #randz=random.randint(0,255)
        lvlColor[manufacture]=(manufactureColors[manufacture][0], manufactureColors[manufacture][1], manufactureColors[manufacture][2])
        
        
        

# print(uniqueModels)

['AT&T' 'Sprint' 'T-Mobile' 'Verizon']
['Alcatel' 'BlackBerry' 'HTC' 'Huawei' 'Kyocera' 'LG' 'Motorola' 'RED'
 'Razer' 'Samsung' 'ZTE' 'CAT' 'Coolpad' 'Essential' 'Google' 'Slate'
 'Sonim' 'ASUS' 'OnePlus' 'Revvl(T-Mobile)' 'Sony' 'Nokia' 'Orbic' 'Palm'
 'Verizon']


In [3]:
# Get same models across carriers
sameModel=[]
sameManu=[]
for model in uniqueModels:
    tempsDF=carrierDF[carrierDF.Model==model]
    if len(tempsDF.Carrier.unique())==4: # all four carriers 
        sameModel.append(model)
        man=list(tempsDF.Manufacture.unique())[0]
        sameManu.append(list(tempsDF.Manufacture.unique())[0])
        
        
print(len(sameModel))
print(sameModel)
        

20
['One M9', 'G5', 'G6', 'G8 ThinQ', 'V20', 'Moto Z (2) Force', 'Galaxy J3 Emerge', 'Galaxy Note Edge', 'Galaxy Note5', 'Galaxy Note8', 'Galaxy Note9', 'Galaxy S5', 'Galaxy S6', 'Galaxy S6 Edge', 'Galaxy S7', 'Galaxy S7 Edge', 'Galaxy S8', 'Galaxy S8+', 'Galaxy S9', 'Galaxy S9+']


In [4]:
# Graph overall latency for each carrier
print(statistics.mean(carrierDF.difference))
print(statistics.median(carrierDF.difference))

uniqueCarrier=carrierDF.Carrier.unique()

# Average each update for one bulletin
perCar=[[] for _ in uniqueCarrier]
perUpdate=[[] for _ in uniqueCarrier]

for bul in carrierDF.Bulletin_Level.unique():
    tmp=carrierDF[carrierDF.Bulletin_Level==bul]
    for index,car in enumerate(uniqueCarrier):
        tmp2=tmp[tmp.Carrier==car]
        if len(tmp2.difference)>1:
            newItem=statistics.mean(tmp2.difference.values)
            perUpdate[index].append(newItem)
            perCar[index].append(car)
        elif len(tmp2.difference)==1:
            newItem=tmp2.difference.values[0]
            perUpdate[index].append(newItem)
            perCar[index].append(car)



data=[]
for index,car in enumerate(uniqueCarrier):
    print(car)
    print(statistics.mean(perUpdate[index]))
    print(statistics.median(perUpdate[index]))
    data.append(go.Box(
        x=perCar[index],
        y=perUpdate[index],
        name=str(car),
        marker=dict(color='rgb'+str(lvlColor[car])),
        showlegend=False,
        boxpoints = 'all',
    )) 

# Edit the layout
layout = dict(#title = "Update Latency across Carriers",
              #xaxis = dict(title = 'Carriers'),
              yaxis = dict(title = 'Days',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              plot_bgcolor='rgba(0,0,0,0)',
              font=dict(size=20)
        )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig)

# pio.write_image(fig, 'avg-patch-delay-by-carrier-per-bulletin.pdf', width=1000, height=600)


24.248964803312628
22.0
AT&T
28
28.0
Sprint
20
22
T-Mobile
17
18
Verizon
21
21.5


In [5]:
data = []

att=[[] for _ in range(2)]
ver=[[] for _ in range(2)]
tmobile=[[] for _ in range(2)]
spt=[[] for _ in range(2)]

matchMan=[]
print("Number of Matching Models: "+str(len(sameModel)))

perCarMan={}
perCarMan['AT&T']={}
perCarMan['Verizon']={}
perCarMan['T-Mobile']={}
perCarMan['Sprint']={}

carrierDF.difference=carrierDF.difference.astype(float)


attC=0
verC=0
sprC=0
tmobC=0


for model in sameModel:
    tempMan=carrierDF[carrierDF.Model==model]
    matchMan.extend(tempMan.Manufacture.unique())
    
#     if matchMan[-1]=="Samsung":
#         attC+=len(tempMan[tempMan.Carrier=='AT&T'])
#         verC+=len(tempMan[tempMan.Carrier=='Verizon'])
#         sprC+=len(tempMan[tempMan.Carrier=='T-Mobile'])
#         tmobC+=len(tempMan[tempMan.Carrier=='Sprint'])
    

    # check att
    newatt=tempMan[tempMan.Carrier=='AT&T']
    att[0].extend(newatt.difference.values)
    att[1].extend(newatt.Manufacture.values)
    if att[1][-1] in perCarMan['AT&T']:
        perCarMan['AT&T'][att[1][-1]]=np.concatenate((perCarMan['AT&T'][att[1][-1]],newatt.difference.values),axis=None)
    else:
        perCarMan['AT&T'][att[1][-1]]=newatt.difference.values
        
    # check verizon
    newver=tempMan[tempMan.Carrier=='Verizon']
    ver[0].extend(newver.difference.values)
    ver[1].extend(newver.Manufacture.values)
    if ver[1][-1] in perCarMan['Verizon']:
        perCarMan['Verizon'][ver[1][-1]]=np.concatenate((perCarMan['Verizon'][ver[1][-1]],newver.difference.values),axis=None)
    else:
        perCarMan['Verizon'][ver[1][-1]]=newver.difference.values
    
    # check tmobile
    newtmb=tempMan[tempMan.Carrier=='T-Mobile']
    tmobile[0].extend(newtmb.difference.values)
    tmobile[1].extend(newtmb.Manufacture.values)
    if tmobile[1][-1] in perCarMan['T-Mobile']:
        perCarMan['T-Mobile'][tmobile[1][-1]]=np.concatenate((perCarMan['T-Mobile'][tmobile[1][-1]],newtmb.difference.values),axis=None)
    else:
        perCarMan['T-Mobile'][tmobile[1][-1]]=newtmb.difference.values
    
    
    # check sprint
    newspt=tempMan[tempMan.Carrier=='Sprint']
    spt[0].extend(newspt.difference.values)
    spt[1].extend(newspt.Manufacture.values)
    if spt[1][-1] in perCarMan['Sprint']:
        perCarMan['Sprint'][spt[1][-1]]=np.concatenate((perCarMan['Sprint'][spt[1][-1]],newspt.difference.values),axis=None)
    else:
        perCarMan['Sprint'][spt[1][-1]]=newspt.difference.values

# calculate the average per shared manufacture and carrier
matchMan=list(set(matchMan))
for man in matchMan:
    print(man)
    print(np.mean(perCarMan['AT&T'][man]))
    print(np.mean(perCarMan['T-Mobile'][man]))
    print(np.mean(perCarMan['Sprint'][man]))
    print(np.mean(perCarMan['Verizon'][man]))
    print()
    
    if man == "Samsung":
        attC+=len(perCarMan['AT&T'][man])
        verC+=len(perCarMan['Verizon'][man])
        sprC+=len(perCarMan['Sprint'][man])
        tmobC+=len(perCarMan['T-Mobile'][man])
    
print("NUMBER OF POINTS FOR SAMSUNG")
print("AT&T: " + str(attC))
print("Verizon: " + str(verC))
print("Sprint: " + str(sprC))
print("T-Mobile: " + str(tmobC))
    
data=[]
data.append(go.Box(
    x=att[1],
    y=att[0],
    name="AT&T",
    marker=dict(
                    color='rgb'+str(lvlColor["AT&T"]),
                )
)) 


data.append(go.Box(
    x=tmobile[1],
    y=tmobile[0],
    name="T-Mobile",
    marker=dict(
                    color='rgb'+str(lvlColor["T-Mobile"]),
                )
))

data.append(go.Box(
    x=spt[1],
    y=spt[0],
    name="Sprint",
    marker=dict(
                    color='rgb'+str(lvlColor["Sprint"]),
                )
))
 
data.append(go.Box(
    x=ver[1],
    y=ver[0],
    name="Verizon",
    marker=dict(
                    color='rgb'+str(lvlColor["Verizon"]),
                )
))    
    
# Edit the layout
layout = dict(#title = "Rollout Latency for Similiar Manufactures Across Carriers",
              #xaxis = dict(title = 'Manufacturers'),
              yaxis = dict(title = 'Days',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              plot_bgcolor='rgba(0,0,0,0)',
              boxmode='group',
              legend_orientation="h",
              legend=dict(x=0.45, y=1.07),
              font=dict(size=20)
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig)

# pio.write_image(fig, 'carrier-manufacture-same-models.pdf', width=1000, height=600)


Number of Matching Models: 20
LG
29.863636363636363
21.289473684210527
21.0625
36.53333333333333

Samsung
28.14814814814815
17.24607329842932
25.015267175572518
24.05

HTC
30.0
16.333333333333332
29.0
78.0

Motorola
35.125
17.142857142857142
47.0
48.25

NUMBER OF POINTS FOR SAMSUNG
AT&T: 162
Verizon: 80
Sprint: 131
T-Mobile: 191


In [ ]:
#for each phone model, calculate the average delay on the same bulletin updated
normalizedDF=[]


for model in uniqueModels:
    tmpDFmod=carrierDF[carrierDF.Model==model]
    for bul in tmpDFmod.Bulletin_Level.unique():
        row=[]
        tmpDFbul=tmpDFmod[tmpDFmod.Bulletin_Level==bul]
        
        if len(tmpDFbul)>1: #then calculate the average
            row.append(tmpDFbul.Manufacture.values[0])
            row.append(tmpDFbul.Model.values[0])
            row.append(tmpDFbul.Bulletin_Level.values[0])
            row.append(tmpDFbul.difference.mean())
            
        else:
            row.append(tmpDFbul.Manufacture.values[0])
            row.append(tmpDFbul.Model.values[0])
            row.append(tmpDFbul.Bulletin_Level.values[0])
            row.append(tmpDFbul.difference.values[0])
        normalizedDF.append(row.copy()) 
            
normalizedDF=pd.DataFrame(normalizedDF, columns=['Manufacture','Model','Bulletin_Level','difference'])         
normalizedDF['Bulletin_Year']=normalizedDF.Bulletin_Level.apply(lambda x: x[:4])


sortedMan=[]
meanMan=[]
for man in uniqueManufactures:
    tmps=normalizedDF[normalizedDF.Manufacture==man]
    if len(tmps.Model.unique())>2: # only include manfuactures with 3 or more phone models
        sortedMan.append(man)
        meanMan.append(tmps.difference.mean())

sortedMan = [x for _,x in sorted(zip(meanMan,sortedMan))]
meanMan = sorted(meanMan)
   
print(sortedMan)
print(meanMan)
print()

data=[]
for man in sortedMan:
    #tmps=carrierDF[carrierDF.Manufacture==man]
    tmps=normalizedDF[normalizedDF.Manufacture==man]
    data.append(go.Box(
        y=tmps.difference.values,
        name=str(man),
        boxpoints = 'all',
        marker=dict(
                    color='rgb'+str(lvlColor[man]),
                ),
        showlegend=False
    ))   
#     print(man)
#     print(len(tmps.difference.values))
#     print()
    
# Edit the layout
layout = dict(#title = "Rollout Latency for Manufactures",
              #xaxis = dict(title = 'Manufacturers'),
              yaxis = dict(title = 'Days',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              plot_bgcolor='rgba(0,0,0,0)',
              font=dict(size=20)
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig)



pio.write_image(fig, 'ALL-manufactures-rollout-latency-avgSameBulletin.pdf', width=1000, height=600)


['Google', 'Revvl(T-Mobile)', 'HTC', 'Sony', 'BlackBerry', 'Samsung', 'LG', 'Alcatel', 'Coolpad', 'ZTE', 'Motorola', 'Verizon', 'ASUS', 'Kyocera']
[2.19672131147541, 14.615384615384615, 19.852941176470587, 20.0, 23.68, 23.78944246737841, 24.240506329113924, 24.764705882352942, 26.5, 27.027027027027028, 31.853846153846153, 35.45454545454545, 37.36363636363637, 46.81481481481482]

